In [1]:
import pandas as pd
import math
import os
import pyperclip
import spotipy
import matplotlib.colors as mcolors
import openpyxl
import numpy as np

from IPython.display import clear_output
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
from openpyxl.styles import PatternFill
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting import Rule
from openpyxl.utils.dataframe import dataframe_to_rows

pd.options.display.float_format = '{:,.0f}'.format

load_dotenv()

True

In [2]:
scope = "user-library-read"

sp = spotipy.Spotify(auth_manager = SpotifyOAuth(scope = scope,
                                                 client_id = os.environ.get("spotifyClientId"),
                                                 client_secret = os.environ.get("spotifyClientSecret"),
                                                 redirect_uri = 'https://example.com/callback'))

In [3]:
# define artists

chumbawamba = sp.artist('0TcYeHEK9sBtv7xPbKhzHz')
silento = sp.artist('7juKTDFlPesGeWQ1GmjmOv')
beatles = sp.artist("3WrFJ7ztbogyGnTHbHJFl2")
taylor_swift = sp.artist('06HL4z0CvFAxyc27GXpf02')
john_cage = sp.artist('1Z3fF5lZdCM0ZHugkGoH8s')
elvis = sp.artist('43ZHCT0cAZBISjO8DG9PnE')
garth_brooks = sp.artist('4BclNkZtAUq1YrYNzye3N7')
ice_cube = sp.artist('3Mcii5XWf6E0lrY3Uky4cA')
beethoven = sp.artist('2wOqMjp9TyABvtHdOSOTUS')
harry_styles = sp.artist('6KImCVD70vtIoJWnq6nGn3')
lou_bega = sp.artist('46lnlnlU0dXTDpoAUmH6Qx')

#clear_output()

Enter the URL you were redirected to: https://example.com/callback?code=AQAA5ZXsq6CPQGeTozKXd7ExV2uUnqMZ8yMBFyvr6iNvyzznBiZvgXsLfitBeBDPclyn64wI6DXKcchavcYKmEZ4tGvU_Q_srhHwXui6igbA5AoF6-eY-RS9bZv7NOXaUIj-vnTXrYSGZiYGi71m2_jESlI_w_6fjSdXTyZx261bKnkirV3g-6kW1QcBUwVPVcDI


In [4]:
artists = [chumbawamba,
           silento,
           beatles,
           taylor_swift,
           john_cage,
           elvis,
           garth_brooks,
           ice_cube,
           beethoven,
           harry_styles,
           lou_bega]

In [5]:
def truncate(s: str, char_length: int = 20) -> str:
    '''
    Truncates a string to char_length and adds `...` to indicate truncation.
    Used to make tables more narrow.
    '''
    return s[0:char_length] + ('...' if len(s) > char_length else '')

def make_top_tracks_df(artist: dict) -> pd.DataFrame:
    '''
    Return DataFrame with top tracks information for an artist.
    '''
    artist_name = artist['name'].replace("Harry", "Harold")
    uri = artist['uri']

    tmp = pd.DataFrame()
    top_tracks = sp.artist_top_tracks(uri)['tracks']
    for track in top_tracks:
        idx = len(tmp)
        tmp.loc[idx, 'artist'] = artist_name
        tmp.loc[idx, 'track'] = truncate(track['name'])
        tmp.loc[idx, 'song-popularity'] = track['popularity']
        tmp.loc[idx, 'album'] = truncate(track['album']['name'], 15)
        seconds = track['duration_ms'] / 1000 
        minutes = math.floor(seconds / 60)
        seconds_remainder = seconds % 60
        tmp.loc[idx, 'duration'] = f'{minutes}:{seconds_remainder:0>2.0f}'
    
    tmp['artist-popularity'] = artist['popularity']
    tmp['artist-url'] = artist['external_urls']['spotify']
        
    return tmp

In [6]:
# make big dataframe
combined = pd.DataFrame()
combined_tex = pd.DataFrame() 
for key, artist in enumerate(artists):
    
    tmp = make_top_tracks_df(artist)
    
    # drop data for a few artists
    if artist['name'] not in ['John Cage', 'Garth Brooks', 'Taylor Swift', 'Ice Cube']:
        tmp = tmp.head(4)
    
    combined = pd.concat([combined, tmp])

    tmp['artist-url'] =  r'\link{' + tmp['artist-url'] + "}{link}"
    tmp['song-popularity'] = tmp['song-popularity'].astype(int)
    tmp = tmp.astype(str)
    if key % 2 == 0:
        tmp = r'\cellcolor{gray!7!white}' + tmp.astype(str)

    combined_tex = pd.concat([combined_tex, tmp])

# Catch special characters
combined_tex = combined_tex.replace("&", r"\&", regex = True)

# don't truncate 
with pd.option_context("max_colwidth", 10000):
    s = combined_tex.to_latex(escape = False,
                              index = None)

In [7]:
pyperclip.copy(s)

In [8]:
# make formatted xlsx

wb = openpyxl.Workbook()
ws = wb.active

for r in dataframe_to_rows(combined, index = False, header = True):
    if r != [None]:
        ws.append(r)
        
for key, artist in enumerate(combined.artist.unique()):
    rgb = 0.3 * np.array(mcolors.to_rgb(f'C{key}')) + 0.7 * np.ones(3)
    hex_color = mcolors.to_hex(rgb).lstrip("#")

    fill = PatternFill(bgColor = hex_color) #, end_color = hex_color, fill_type = 'solid')
    dxf = DifferentialStyle(fill = fill)
    rule = Rule(type = "containsText",
                operator = "equal",
                text = artist[0:5], 
                dxf = dxf)
    ws.conditional_formatting.add('A1:A100', rule)

ws.auto_filter.ref = ws.calculate_dimension() # add filter
wb.save("spotify-data.xlsx")